In [1]:
import numpy as np
import nibabel as nib 
import os 
import pandas as pd
from utils.utils import * 
from utils.flips import get_prefered_xy
import math

In [2]:
rois_long = {'V1v': 1, 'V1d': 2, 'V2v': 3, 'V2d': 4, 'V3v': 5, 'V3d': 6, 'hV4': 7, 'VO-1': 8,
 'VO-2': 9, 'PHC-1': 10, 'PHC-2': 11, 'LO-1': 12, 'LO-2': 13, 'TO-1': 14, 'TO-2': 15
}
sior_long = {v: k for k,v in rois_long.items()}
to_reduce = {'V1v': 'V1', 'V1d': 'V1', 'V2v': 'V2', 'V2d': 'V2', 'V3v': 'V3', 'V3d': 'V3'}
sior = {v: k for k,v in rois.items()}

In [4]:
cos_sin = get_prefered_xy(subj_list, sessions, fetch_conds=False) # save that and open it matlab 


(907,)
[[-0.48904547 -0.6321048 ]
 [ 0.05454651  0.70571345]
 [-0.43584672 -0.58906263]
 ...
 [-0.5720981  -0.5707307 ]
 [ 0.4883913   0.5600432 ]
 [ 0.3847739   0.46473083]]
[[ 0.00806928 -0.7572817 ]
 [ 0.4076677  -0.40104052]
 [-0.61324996 -0.56195503]
 ...
 [ 0.28770244 -0.6456871 ]
 [-0.6414153   0.29659677]
 [ 0.40534177  0.01031652]]
[[ 0.01625607 -0.57528704]
 [ 0.18775472  0.66169876]
 [-0.7920724  -0.23686317]
 ...
 [-0.76466393 -0.2328792 ]
 [ 0.4752489   0.67938405]
 [-0.07587725 -0.39778942]]
[[-0.43153003 -0.0264073 ]
 [-0.07680967 -0.53455365]
 [ 0.6410322  -0.48972103]
 ...
 [-0.6190398  -0.5749555 ]
 [ 0.4271968  -0.4626026 ]
 [-0.20331939 -0.11802859]]
[[-0.26468593 -0.02610916]
 [-0.01554198  0.82191086]
 [-0.7282502  -0.34517598]
 ...
 [-0.19735079 -0.6040885 ]
 [-0.54201376  0.6034352 ]
 [-0.48633546  0.40079314]]
[[ 0.04787785  0.75354695]
 [ 0.12887517  0.49418166]
 [-0.7183236   0.14104222]
 ...
 [ 0.25027773 -0.60873365]
 [-0.28760958  0.47812727]
 [-0.67110175

In [6]:
cos_sin  #Y's sould probably be flipped

array([[ 1.        ,  0.        ],
       [ 0.65416318,  0.75635338],
       [ 0.36456782,  0.93117684],
       [ 0.53622568,  0.84407467],
       [ 0.68345046, -0.72999692],
       [ 0.72568923,  0.68802267],
       [ 0.42723933,  0.90413857],
       [-0.29617727,  0.95513296]])

In [10]:
models_subs = {}
models = ['woself']
columns = ["x0", "y0", "sigma", "slope", "intercept", "test_var_explained", "var_explained", "mds_ecc", "mds_ang", "roi", "fit_with"]
model_study = 'woself'
mode = "train"
rotated = True

#cos_sin = get_prefered_xy(subj_list) # save that and open it matlab 
#print(cos_sin)


for i, sub in enumerate(subj_list):
    models_subs[sub] = {}
    for m in models:
        if rotated:
            m_file = os.path.join(models_dir, f'best_fits_{m}_{sub}_{mode}_bestroi.npy')
        models_subs[sub][m] = pd.DataFrame(np.load(m_file, allow_pickle=True), columns=columns)
        models_subs[sub][m][columns[:-2]] = models_subs[sub][m][columns[:-2]].astype(np.float32)
        models_subs[sub][m]['x0_prefered'] = models_subs[sub][m]['x0'] * cos_sin[i, 0]-models_subs[sub][m]['y0'] * cos_sin[i, 1] # the mds have different dimension -> so problem 
        models_subs[sub][m]['y0_prefered'] = models_subs[sub][m]['x0'] * cos_sin[i, 1]+models_subs[sub][m]['y0'] * cos_sin[i, 0]
       # models_subs[sub][m]['distance'] = np.sqrt(models_subs[sub][m]['x0']**2 + models_subs[sub][m]['y0']**2)
   #     models_subs[sub][m]['distance_prefered'] = np.sqrt(models_subs[sub][m]['x0_prefered']**2 + models_subs[sub][m]['y0_prefered']**2)
    sub_m_study = models_subs[sub][model_study]

hier_index = pd.MultiIndex.from_arrays(
    [
        np.tile(list(rois.keys()), len(models_subs)),
        np.repeat(list(models_subs.keys()), len(rois.keys())),
    ],
    names =["roi", "sub"]
)
models_subs["subj01"]["woself"]["test_var_explained"]

# check distribution of values in VO-1s for common conditions 

models_subs

{'subj01': {'woself':              x0        y0     sigma      slope  intercept  test_var_explained  \
  0     -0.609883  1.050000  6.643504  23.417582 -23.205332            0.007374   
  1     -0.586327  1.050000  3.009087   4.445218  -4.261150            0.005575   
  2     -0.602788  1.050000  3.582304   4.870206  -4.725228            0.003986   
  3     -0.681538  1.050000  4.523826   9.787571  -9.576628            0.006088   
  4     -0.769511  1.050000  5.153516  11.236902 -11.024564            0.005520   
  ...         ...       ...       ...        ...        ...                 ...   
  33792 -0.139656 -0.175641  0.015894   1.447698  -0.007107           -0.000779   
  33793 -1.049968  0.657002  3.931461   5.360149  -5.099864            0.016892   
  33794 -1.049875  0.881139  4.426235   4.877195  -4.688420            0.005650   
  33795 -1.050000  0.335569  6.990169  13.822908 -13.599826            0.011453   
  33796 -1.049999  0.494084  4.557807   5.309813  -5.112263        

In [7]:
hemis = ['lh', 'rh']
fields = columns
fields.append('x0_prefered')
fields.append('y0_prefered')
if rotated:
    rotation = "rotated"
if not rotated: 
    rotation = "notrotated"


for model in models:
    for sub, sessions in subjects_sessions.values():
        m = models_subs[sub][model]
        print(f'------ EXPORTING MODEL FOR {sub}')
        for f in fields:
            print(f'\t{f}')
            s = -1
            for hemi in hemis:
                print(f'\t\t{hemi}')
                
                if sub == 'subj06' or sub == 'subj08':
                    maskdata_long_file = os.path.join(data_dir, 'mask', sub, f'{hemi}.{sub}.nans.testrois.mgz')
                else:
                    maskdata_long_file = os.path.join(data_dir, 'mask', sub, f'{hemi}.{sub}.testrois.mgz')
                maskdata_long = nib.load(maskdata_long_file).get_fdata().squeeze()
              #  print(maskdata_long.shape)
        
                maskdata_file = os.path.join(data_dir, 'mask', sub, f'{hemi}.short.reduced.{sub}.testrois.npy')
                maskdata = np.load(maskdata_file, allow_pickle=True).astype(int)

                data_out_file = os.path.join(results_dir, sub, f'{hemi}.{model}.{f}.{rotation}.mgz')
                label_out_file = os.path.join(label_dir,'freesurfer', sub, 'label', f'{hemi}.{model}.{f}.{rotation}.mgz') # annoying but necessary for matlab output
                if os.path.exists(data_out_file):
                    pass
                data_out = np.zeros(maskdata_long.shape)
             
                wrongs = []
                for i in range(maskdata_long.shape[0]):
                    voxel = int(maskdata_long[i])
                    if voxel==0 or voxel > 15:
                        continue
                    s += 1
                    if sior_long[voxel] in to_reduce.keys():
                        roi = rois[to_reduce[sior_long[voxel]]]
                    else:
                        roi = rois[sior_long[voxel]]
                  #  roi = rois[sior[voxel]]
                    # prove that the order is maintained
                    if roi != rois[m.iloc[s].roi]:
                        print(f'{i}: {roi} != {rois[m.iloc[s].roi]}')
                        wrongs.append(i)
                    if f in ['roi', 'fit_with']:
                        data_out[i] = rois[m.iloc[s][f]]
                    elif f == 'mds_ang':
                        data_out[i] = math.degrees(m.iloc[s][f]) + 180
                    elif f == 'intercept':
                        data_out[i] = m.iloc[s][f] + abs(m[f].min()) + 1e-08
                    elif f in ['x0', 'y0', 'x0_prefered', 'y0_prefered']:
                        data_out[i] = m.iloc[s][f] + 1.1
                    elif f in ['var_explained', 'test_var_explained']:
                        data_out[i] = m.iloc[s][f]  + 1 # Add 1 for the variance explain, so we can still mask the non relevant value and plot the "negative" variance
                    else:
                        data_out[i] = m.iloc[s][f]  
                img = nib.Nifti1Image(np.expand_dims(data_out, axis=(1, 2)), np.eye(4))

                nib.loadsave.save(img, data_out_file)
                nib.loadsave.save(img, label_out_file)
                if f == 'mds_ang':
                    for ang in [45, 90, 135, 180, 225, 270, 305]:
                        data_out_file = os.path.join(results_dir, sub, f'{hemi}.{model}.f.{str(ang)}.{rotation}.mgz')
                        label_out_file = os.path.join(label_dir, 'freesurfer', sub, 'label', f'{hemi}.{model}.{f}.{str(ang)}.{rotation}.mgz')
                        data_out_ang = np.copy(data_out)
                        data_out_ang[data_out != 0] += ang
                        data_out_ang = data_out_ang % 360
                        img = nib.Nifti1Image(np.expand_dims(data_out_ang, axis = (1, 2)), np.eye(4))
                        nib.loadsave.save(img, data_out_file)
                        nib.loadsave.save(img, label_out_file)


print('end')
                        
            

------ EXPORTING MODEL FOR subj01
	x0
		lh


/home/harveylab/miniconda3/envs/thesis/lib/python3.10/site-packages/nibabel/spatialimages.py:558: UserWarning: Using large vector Freesurfer hack; header will not be compatible with SPM or FSL
  hdr.set_data_shape(shape)


		rh
	y0
		lh
		rh
	sigma
		lh
		rh
	slope
		lh
		rh
	intercept
		lh
		rh
	test_var_explained
		lh
		rh
	var_explained
		lh
		rh
	mds_ecc
		lh
		rh
	mds_ang
		lh
		rh
	roi
		lh
		rh
	fit_with
		lh
		rh
	x0_prefered
		lh
		rh
	y0_prefered
		lh
		rh
------ EXPORTING MODEL FOR subj02
	x0
		lh
		rh
	y0
		lh
		rh
	sigma
		lh
		rh
	slope
		lh
		rh
	intercept
		lh
		rh
	test_var_explained
		lh
		rh
	var_explained
		lh
		rh
	mds_ecc
		lh
		rh
	mds_ang
		lh
		rh
	roi
		lh
		rh
	fit_with
		lh
		rh
	x0_prefered
		lh
		rh
	y0_prefered
		lh
		rh
------ EXPORTING MODEL FOR subj03
	x0
		lh
		rh
	y0
		lh
		rh
	sigma
		lh
		rh
	slope
		lh
		rh
	intercept
		lh
		rh
	test_var_explained
		lh
		rh
	var_explained
		lh
		rh
	mds_ecc
		lh
		rh
	mds_ang
		lh
		rh
	roi
		lh
		rh
	fit_with
		lh
		rh
	x0_prefered
		lh
		rh
	y0_prefered
		lh
		rh
------ EXPORTING MODEL FOR subj04
	x0
		lh
		rh
	y0
		lh
		rh
	sigma
		lh
		rh
	slope
		lh
		rh
	intercept
		lh
		rh
	test_var_explained
		lh
		rh
	var_explained
		lh
		r